In [40]:
from opentree import OT
import dendropy as dp
import re, os, copy, shutil
from ipywidgets import interactive, FloatSlider
from IPython.display import display, Javascript
import ipywidgets as widgets
import functools
from ipywidgets import VBox, HBox
import pandas as pd
wdir=0
inputfile=0
outputfile=0
os.getcwd()

'/Users/ivan/Documents/Raleigh_lab/dnds'

In [41]:
#modified from https://www.geeksforgeeks.org/remove-invalid-parentheses/
# Python3 program to remove invalid parenthesis
 
# Method checks if character is parenthesis(open
# or closed)
def isParenthesis(c):
    return ((c == '(') or (c == ')'))
 
# method returns true if contains valid
# parenthesis
def isValidString(str):
    cnt = 0
    for i in range(len(str)):
        if (str[i] == '('):
            cnt += 1
        elif (str[i] == ')'):
            cnt -= 1
        if (cnt < 0):
            return False
    return (cnt == 0)
     
# method to remove invalid parenthesis
def removeInvalidParenthesis(str):
    if (len(str) == 0):
        return
         
    # visit set to ignore already visited
    visit = set()
     
    # queue to maintain BFS
    q = []
    temp = 0
    level = 0
     
    # pushing given as starting node into queu
    q.append(str)
    visit.add(str)
    while(len(q)):
        str = q[0]
        q.pop()
        if (isValidString(str)):
            #print(str)
             
            # If answer is found, make level true
            # so that valid of only that level
            # are processed.
            level = True
        if (level):
            continue
        for i in range(len(str)):
            if (not isParenthesis(str[i])):
                continue
                 
            # Removing parenthesis from str and
            # pushing into queue,if not visited already
            temp = str[0:i] + str[i + 1:]
            if temp not in visit:
                q.append(temp)
                visit.add(temp)
    return str

In [42]:
#speciesfile- excel

def make_tree():
    os.chdir(wdir)
    otts=[]
    names_df=pd.read_csv(inputfile)
    names=names_df['Scientific name'].tolist()
    print(names)
    
    for i in range(len(names)):
        otts.append(OT.get_ottid_from_name(names[i]))
    ret = OT.synth_induced_tree(ott_ids = otts,
                                   label_format="name")
    treefile = "unstripped.txt"
    ret.tree.write(path = treefile, schema = "newick")
    ret.tree.print_plot(width=100)
    stripped_tree = dp.Tree.get(file=open("unstripped.txt"), schema="newick")
    taxon_names = [s.label for s in stripped_tree.taxon_namespace]
    # Extract tree with labels.  This will remove all of the empty singleton 
    # entries that otl brings down. 
    final_tree = stripped_tree.extract_tree_with_taxa_labels(taxon_names)
    file1 = open("cypriniformes_speciestree.txt","w")#write mode
    file1.write(str(final_tree))
    file1.close()
    str_tree=str(final_tree)
    #print('string tree:  \n',str_tree)
    
    
    for_brackets=re.split('(\W+)',str_tree)
    count=-1
    for i in for_brackets:
        count=count+1
        if i=='':
            for_brackets.pop(count)
        if 'mrcaott' in i:
            for_brackets.pop(count)

    count=-1
    for i in for_brackets:
        count=count+1
        #print(i[0].isupper())
        if i[0].isupper()==False and '_' in i:
            for_brackets.pop(count)
    #print(for_brackets)
            
    count=-1
    for i in names:
        count=count+1
        j=i.replace(' ','_')
        names[count]=j
    #print('new names:{} \n'.format(len(names)),names)
    
    count=-1
    good_tree=[]
    for i in for_brackets:
        count=count+1
        if '_' in i:
            for j in names:
                if i[0:10] in j:
                    good_tree.append(j)
                    #print('appending {}'.format(j))
                    
                    
        else:
            if 'Pan' in i:
                good_tree.append('Pan_Troglo')
            if '(' not in i and ')' not in i and ',' not in i:
                print ('deleting {}'.format(i))
            else: 
                good_tree.append(i)
                #print('appending {}'.format(i))
    good_tree=''.join(good_tree)
    good_tree=good_tree.replace(',)','')
    good_tree=good_tree.replace('()','')
    good_tree=removeInvalidParenthesis(good_tree)
    print(good_tree)
    with open(outputfile,'w') as text_file:
        text_file.write(good_tree)
    return good_tree
    
    

In [43]:
def run_next(ev,button):
    global wdir, inputfile, outputfile
    wdir=wdir_w.value
    inputfile=inputfile_w.value
    outputfile=outputfile_w.value
        
    display(Javascript('IPython.notebook.execute_cells([IPython.notebook.get_selected_index()+1])'))

wdir_w=widgets.Text(value='/Users/ivan/Documents/Raleigh_lab/dnds', description= 'directory with species file')
inputfile_w=widgets.Text(value='IAPP_orthologs.csv', description= 'species csv file')
outputfile_w=widgets.Text(value='species_tree_IAPP_procedural', description= 'output file name')

button_js=widgets.Button(description="Build tree")
button_js.on_click(functools.partial(run_next,button=button_js))

box=HBox([wdir_w,inputfile_w,outputfile_w,button_js])
#print'''Please input the path to the directory containing the allignment
#Note that it must be as /Users/name/directory rather than ~/directory
                          
#Output file can have any name, python will creatte it for you

#Currently only takes .phy files, 10 char trimmed (not padded)

#Press 'Convert file' and the output .txt will appear in your specified directory '''
display(box)

<IPython.core.display.Javascript object>

In [44]:
make_tree()

['Bos taurus', 'Canis lupus familiaris', 'Equus caballus', 'Felis catus', 'Homo sapiens', 'Loxodonta africana', 'Macaca mulatta', 'Microcebus murinus', 'Monodelphis domestica', 'Mus musculus', 'Oryctolagus cuniculus', 'Ovis aries', 'Pan troglodytes', 'Phascolarctos cinereus', 'Pteropus alecto', 'Pteropus vampyrus', 'Rattus norvegicus', 'Rattus rattus', 'Sarcophilus harrisii', 'Sus scrofa']
                                         /------------+-+-+-+-+-+-+-+-+-+-+-- Mus musculus          
            /-+-+-+-+-+-+--+-+-+-+-+-+-+-+                                                          
            |                            |                /-+-+-+-+-+-+-+-+-- Rattus rattus         
      /-+-+-+                            \-+-+-+-+--+-+-+-+                                         
      |     |                                             \-------------+-+-- Rattus norvegicus     
      |     |                                                                                       
 

'((((((Mus_musculus,(Rattus_rattus,Rattus_norvegicus)),Oryctolagus_cuniculus),((Macaca_mulatta,(Homo_sapiens,Pan_Troglo)),Microcebus_murinus)),(((((Ovis_aries,Bos_taurus),Sus_scrofa),Equus_caballus),(Pteropus_alecto,Pteropus_vampyrus)),(Canis_lupus_familiaris,Felis_catus))),Loxodonta_africana),((Phascolarctos_cinereus,Sarcophilus_harrisii),Monodelphis_domestica))'